# Ejercicio 3 – Predicción de serie temporal horaria con Prophet

---

### Paso 1: Preparar el dataset

In [ ]:
from prophet import Prophet
import pandas as pd

# Cargar dataset desde archivo CSV
df = pd.read_csv("data/m5_consumo_energia.csv", parse_dates=["dt"])

# Establecer columna temporal como índice
df.set_index("dt", inplace=True)

df.info()

In [ ]:
df.describe()

In [ ]:
import matplotlib.pyplot as plt
# Visualizar las variables clave
df.plot(subplots=True, figsize=(10, 8), grid=True)
plt.tight_layout()
plt.show()

Seleccionamos solo el mes de Julio para facilitar el análisis

In [ ]:
daily_df = df[df.index.month == 7]
daily_df.info()

In [ ]:
daily_df.describe()

In [ ]:
# Visualizar las variables clave
daily_df.plot(subplots=True, figsize=(10, 8), grid=True)
plt.tight_layout()
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Calcular matriz de correlación
corr = daily_df.corr()

# Representar heatmap
plt.figure(figsize=(6, 6))
sns.heatmap(corr, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Matriz de correlación entre variables")
plt.tight_layout()
plt.show()

### Paso 2: Separar datos de entrenamiento y validación
Dividimos el dataset en entrenamiento y validación

In [ ]:
train_df = daily_df[daily_df.index.day <= 29]
val_df = daily_df[daily_df.index.day > 29]

# Prophet requiere columnas: 'ds' para fecha y 'y' para valor
train_df = train_df.rename(columns={"Energia Consumida": "y"})
val_df = val_df.rename(columns={"Energia Consumida": "y"})

In [ ]:
print("Entrenamiento: {} muestras".format(len(train_df)))
print("Validación: {} muestras".format(len(val_df)))

### Paso 2: Añadir regresores al dataframe

In [ ]:
# Preparar dataset con variables externas (Temperatura y Humedad)
train_reg_df = train_df.reset_index().rename(columns={
    "dt": "ds",
    "Energia Consumida": "y"
})
train_reg_df = train_reg_df[["ds","y"]]

# Añadir regresores
train_reg_df["temperatura"] = train_df["Temperatura"].values
train_reg_df["humedad"] = train_df["Humedad"].values

train_reg_df

---

### Paso 3: Definir modelo con regresores

In [ ]:
model = Prophet()
# model.add_regressor("temperatura")
# model.add_regressor("humedad")

model.fit(train_reg_df)

---

### Paso 3: Crear futuro con regresores

In [ ]:
# Reconstruimos el dataset completo
all_df = pd.concat([train_df, val_df], axis=0)
all_df = all_df.rename({"Temperatura":"temperatura",
                        "Humedad":"humedad",}, axis=1)


In [ ]:
# Duplicamos y eliminamos la variable a predecir
future_df = all_df.copy()
future_df = future_df.drop(["y"], axis=1)
future_df['ds'] = future_df.index
future_df

---

### Paso 4: Predecir y visualizar

In [ ]:
forecast_reg = model.predict(future_df)

# Visualización
fig1 = model.plot(forecast_reg)
fig2 = model.plot_components(forecast_reg)

In [ ]:
import matplotlib.pyplot as plt

train_df['ds'] = train_df.index
val_df['ds'] = val_df.index
# También puedes obtener el final de train como referencia
cutoff_date = train_df['ds'].max()

# Filtramos la parte futura del forecast
forecast_reg_future = forecast_reg[forecast_reg["ds"] > cutoff_date]

# Graficamos todo
plt.figure(figsize=(12, 6))

# Serie real
plt.plot(train_df["ds"], train_df["y"], label="Entrenamiento", color="blue")
plt.plot(val_df["ds"], val_df["y"], label="Validación", color="green")

# Predicción
plt.plot(forecast_reg_future["ds"], forecast_reg_future["yhat"], label="Predicción Prophet", color="orange", linestyle="--")

# Intervalo de confianza
plt.fill_between(forecast_reg_future["ds"],
                 forecast_reg_future["yhat_lower"],
                 forecast_reg_future["yhat_upper"],
                 color="blue", alpha=0.2, label="Intervalo 95%")

# Formato
plt.title("Predicción de Prophet vs serie real")
plt.xlabel("Fecha")
plt.ylabel("Consumo (kWh)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Graficamos todo
plt.figure(figsize=(12, 6))

# Serie real
plt.plot(train_df[-96:]["ds"], train_df[-96:]["y"], label="Entrenamiento", color="blue")
plt.plot(val_df["ds"], val_df["y"], label="Validación", color="green")

# Predicción
plt.plot(forecast_reg_future["ds"], forecast_reg_future["yhat"], label="Predicción Prophet", color="orange", linestyle="--")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

---

### Paso 5: Evaluar el modelo

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

mae = mean_absolute_error(val_df["y"], forecast_reg_future["yhat"])
rmse = np.sqrt(mean_squared_error(val_df["y"], forecast_reg_future["yhat"]))

print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")

---

## ✅ Resultado esperado

- Preparar tus datos en formato adecuado para Prophet.
- Entrenar modelos con y sin regresores externos.
- Visualizar tendencias, estacionalidades y componentes.
- Comparar predicciones y detectar el efecto de variables como temperatura o humedad.